## ***Visión por Computadora - 2024***
### *Lista 01 - Ejercicio 5*

Stefano Aragoni, Carol Arévalo, José González, Luis Santos

-----------


5. Implementar un algoritmo de segmentación para imágenes RGB, utilizando de fondo un algoritmo de k-medias. Se debe permitir al usuario elegir el parámetro k ≥ 2 del número de segmentos que desea obtener. Mostrar con varios ejemplos (buenos y malos) los alcances y limitaciones de este algoritmo. Para cada caso, mostrar

    - la imagen original
    - el mapa de segmentos o clases
    - la imagen cuantizada (promedio de color en cada segmento).


Referencias:

- https://www.kdnuggets.com/2019/08/introduction-image-segmentation-k-means-clustering.html
- https://www.geeksforgeeks.org/image-segmentation-using-k-means-clustering/
- https://medium.com/swlh/image-segmentation-using-k-means-clustering-46a60488ae71
- https://www.sciencedirect.com/science/article/pii/S1877050915014143
- https://medium.com/mlearning-ai/k-means-clustering-71a875dbce3c

In [5]:
# Implementación de un algoritmo de segmentación para imágenes RGB utilizando el método de k-medias sin el uso de librerías externas.

# Importar las bibliotecas necesarias
import numpy as np
import cv2

# Función para inicializar los centroides de manera aleatoria
def inicializar_centroides(datos, k):
    indices = np.random.choice(len(datos), k, replace=False)
    centroides = datos[indices]
    return centroides

# Función para asignar cada punto al centroide más cercano
def asignar_puntos_a_centroides(datos, centroides):
    distancias = np.linalg.norm(datos - centroides[:, np.newaxis], axis=2)
    etiquetas = np.argmin(distancias, axis=0)
    return etiquetas

# Función para actualizar la posición de los centroides
def actualizar_centroides(datos, etiquetas, k):
    centroides_actualizados = np.array([datos[etiquetas == i].mean(axis=0) for i in range(k)])
    return centroides_actualizados

# Función principal para el algoritmo de k-medias
def k_medias(imagen, k, max_iter=100):
    # Obtener los datos de la imagen en un formato adecuado
    datos = imagen.reshape((-1, 3)).astype(np.float32)

    # Inicializar los centroides
    centroides = inicializar_centroides(datos, k)

    # Iterar para ajustar los centroides
    for _ in range(max_iter):
        # Asignar puntos a centroides
        etiquetas = asignar_puntos_a_centroides(datos, centroides)

        # Actualizar los centroides
        centroides_actualizados = actualizar_centroides(datos, etiquetas, k)

        # Verificar convergencia
        if np.all(centroides == centroides_actualizados):
            break

        centroides = centroides_actualizados

    # Crear el mapa de segmentos
    mapa_segmentos = centroides[etiquetas].reshape(imagen.shape)

    # Crear la imagen cuantizada
    imagen_cuantizada = np.zeros_like(imagen)
    for i in range(k):
        imagen_cuantizada[etiquetas == i] = centroides[i]

    return imagen, mapa_segmentos, imagen_cuantizada

# Ejemplo de uso
if __name__ == "__main__":
    # Cargar una imagen de ejemplo (reemplazar con la ruta de la imagen deseada)
    ruta_imagen = "imgs/PRI192044821.jpg"
    imagen_original = cv2.imread(ruta_imagen)
    imagen_original = cv2.cvtColor(imagen_original, cv2.COLOR_BGR2RGB)

    # Parámetro k proporcionado por el usuario
    k_deseado = 4

    # Aplicar el algoritmo de k-medias
    imagen_resultado, mapa_segmentos, imagen_cuantizada = k_medias(imagen_original, k_deseado)

    # Mostrar resultados
    plt.figure(figsize=(10, 5))

    plt.subplot(131)
    plt.imshow(imagen_resultado)
    plt.title('Imagen Original')

    plt.subplot(132)
    plt.imshow(mapa_segmentos)
    plt.title('Mapa de Segmentos')

    plt.subplot(133)
    plt.imshow(imagen_cuantizada)
    plt.title('Imagen Cuantizada')

    plt.show()


AttributeError: 'NoneType' object has no attribute 'reshape'